In [102]:
# Imports
import pandas as pd

Baseball Savant

In [103]:
# Loading Data
hitting = pd.read_csv('baseball_savant_hitting.csv').drop(columns=['fast_swing_rate', 'avg_swing_speed', 'blasts_contact', 'blasts_swing', 'squared_up_contact', 'squared_up_swing',]).dropna()
pitching = pd.read_csv('baseball_savant_pitching.csv').drop(columns=['p_opp_batting_avg', 'p_opp_on_base_avg'])

In [104]:
fourseam_columns = ['n_ff_formatted', 'ff_avg_speed', 'ff_avg_spin', 'ff_avg_break_x', 'ff_avg_break_z']
slider_columns = ['n_sl_formatted', 'sl_avg_speed', 'sl_avg_spin', 'sl_avg_break_x','sl_avg_break_z']
changeup_columns = ['n_ch_formatted', 'ch_avg_speed', 'ch_avg_spin', 'ch_avg_break_x', 'ch_avg_break_z']
curveball_columns = ['n_cu_formatted', 'cu_avg_speed', 'cu_avg_spin', 'cu_avg_break_x', 'cu_avg_break_z']
sinker_columns = ['n_si_formatted', 'si_avg_speed', 'si_avg_spin', 'si_avg_break_x', 'si_avg_break_z']
cutter_columns = ['n_fc_formatted', 'fc_avg_speed', 'fc_avg_spin', 'fc_avg_break_x', 'fc_avg_break_z']

# Splitting pitching data by pitch type
fourseams = pitching.drop(columns = slider_columns + changeup_columns + curveball_columns + sinker_columns + cutter_columns).dropna(subset=fourseam_columns)
sliders = pitching.drop(columns = fourseam_columns + changeup_columns + curveball_columns + sinker_columns + cutter_columns).dropna(subset=slider_columns)
changeups = pitching.drop(columns = fourseam_columns + slider_columns + curveball_columns + sinker_columns + cutter_columns).dropna(subset=changeup_columns)
curveballs = pitching.drop(columns = fourseam_columns + slider_columns + changeup_columns + sinker_columns + cutter_columns).dropna(subset=curveball_columns)
sinkers = pitching.drop(columns = fourseam_columns + slider_columns + changeup_columns + curveball_columns + cutter_columns).dropna(subset=sinker_columns)
cutters = pitching.drop(columns = fourseam_columns + slider_columns + changeup_columns + curveball_columns + sinker_columns).dropna(subset=cutter_columns)

# n_type_formatted * pa = relative weight for how many pitches of this type the pitcher threw (might be useful for weighted averages or weighted correlations)
fourseams['ff_weight'] = fourseams['n_ff_formatted'] * fourseams['pa'] / 100
sliders['sl_weight'] = sliders['n_sl_formatted'] * sliders['pa'] / 100
changeups['ch_weight'] = changeups['n_ch_formatted'] * changeups['pa'] / 100
curveballs['cu_weight'] = curveballs['n_cu_formatted'] * curveballs['pa'] / 100
sinkers['si_weight'] = sinkers['n_si_formatted'] * sinkers['pa'] / 100
cutters['fc_weight'] = cutters['n_fc_formatted'] * cutters['pa'] / 100


Kaggle - Aggregating by Team

In [ ]:
# Batting Data - see readme.txt for description of batting stats
team_batting = pd.read_csv('Kaggle/Batting.csv')[['teamID', 'yearID', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF']].dropna().groupby(['teamID', 'yearID']).sum()
team_batting['PA'] = team_batting['AB'] + team_batting['BB'] + team_batting['SF'] + team_batting['SH'] + team_batting['HBP']
team_batting['AVG'] = team_batting['H'] / team_batting['AB']
team_batting['OBP'] = (team_batting['BB'] + team_batting['HBP'] + team_batting['H']) / team_batting['PA']
team_batting['SLG'] = (team_batting['H'] + team_batting['2B'] + 2 * team_batting['3B'] + 3 * team_batting['HR']) / team_batting['AB']
team_batting['OPS'] = team_batting['OBP'] + team_batting['SLG']

In [ ]:
# Pitching Data - just ERA
team_pitching = pd.read_csv('Kaggle/Pitching.csv')[['teamID', 'yearID', 'ER', 'IPouts']].dropna().groupby(['teamID', 'yearID']).sum()
team_pitching['ERA'] = 27 * team_pitching['ER'] / team_pitching['IPouts']

In [ ]:
# Salary Data
team_salaries = pd.read_csv('Kaggle/Salaries.csv')[['teamID', 'yearID', 'salary']].dropna().groupby(['teamID', 'yearID']).sum()
year_salaries = team_salaries.groupby('yearID').mean()
team_salaries = team_salaries.join(year_salaries, on='yearID', rsuffix='_league_avg')
team_salaries['salary/avg_salary'] = team_salaries['salary'] / team_salaries['salary_league_avg'] # comparing team salary to average team salary that year

In [246]:
# General Team Data
teams = pd.read_csv('Kaggle/Teams.csv')[['yearID', 'teamID', 'name', 'attendance', 'G', 'W', 'L', 'R', 'RA']].dropna()
teams = teams.groupby(['teamID', 'yearID']).sum() # Sum isn't necesary with one piece of data, but it gets table into samed group format as others
teams['W%'] = teams['W'] / teams['G'] # Win rate

# Joining tables together
aggregate_teams = team_batting.join(team_pitching).join(team_salaries).join(teams)
aggregate_teams['avg_runs_scored'] = aggregate_teams['R'] / aggregate_teams['G']
aggregate_teams['avg_runs_allowed'] = aggregate_teams['RA'] / aggregate_teams['G']
aggregate_teams = aggregate_teams[['name', 'AVG', 'OBP', 'SLG', 'OPS', 'avg_runs_scored', 'ERA', 'avg_runs_allowed', 'salary', 'salary/avg_salary', 'G', 'W%']].dropna()
aggregate_teams

name       AVG       OBP       SLG       OPS  \
teamID yearID                                                                 
ANA    1997          Anaheim Angels  0.272033  0.343354  0.416311  0.759665   
       1998          Anaheim Angels  0.271758  0.332590  0.415453  0.748043   
       1999          Anaheim Angels  0.255552  0.319361  0.394976  0.714337   
       2000          Anaheim Angels  0.279673  0.349757  0.472459  0.822216   
       2001          Anaheim Angels  0.260674  0.324385  0.404972  0.729357   
...                             ...       ...       ...       ...       ...   
WAS    2011    Washington Nationals  0.242419  0.304433  0.382650  0.687084   
       2012    Washington Nationals  0.261443  0.319563  0.427605  0.747167   
       2013    Washington Nationals  0.251104  0.309079  0.397535  0.706614   
       2014    Washington Nationals  0.253158  0.317889  0.392999  0.710888   
       2015    Washington Nationals  0.251105  0.318130  0.402542  0.720672   

               avg_runs_scored       ERA  avg_runs_allowed       salary  \
teamID yearID                                                             
ANA    1997           5.117284  4.516499          4.901235   31135472.0   
       1998           4.858025  4.487535          4.833333   41281000.0   
       1999           4.388889  4.791337          5.098765   55388166.0   
       2000           5.333333  5.015884          5.364198   51464167.0   
       2001           4.265432  4.200556          4.506173   47535167.0   
...                        ...       ...               ...          ...   
WAS    2011           3.875776  3.583027          3.993789   63856928.0   
       2012           4.512346  3.340522          3.666667   80855143.0   
       2013           4.049383  3.585889          3.864198  113703270.0   
       2014           4.234568  3.029238          3.425926  131983680.0   
       2015           4.339506  3.619656          3.919753  155587472.0   

               salary/avg_salary    G        W%  
teamID yearID                                    
ANA    1997             0.773356  162  0.518519  
       1998             0.968823  162  0.524691  
       1999             1.112042  162  0.432099  
       2000             0.926651  162  0.506173  
       2001             0.727333  162  0.462963  
...                          ...  ...       ...  
WAS    2011             0.687989  161  0.496894  
       2012             0.827095  162  0.604938  
       2013             1.124096  162  0.530864  
       2014             1.240325  162  0.592593  
       2015             1.328240  162  0.512346  

[888 rows x 12 columns]